In [132]:
import numpy as np
import pandas as pd
import scipy.optimize as sco
from scipy.optimize import basinhopping
import random
import warnings

warnings.filterwarnings("ignore")

In [133]:
data = {'ISIN코드':['US88339J1051','US8522341036','US2521311074','US64125C1099','US98980G1022','US29786A1060','US98138H1014',
                    'US5486611073','US0970231058','US6200763075','현금','합계'],
        '종목명':['트레이드 데스크(TTD)','블록(스퀘어)(SQ)','덱스컴','NEUROCRINE BIOSCIENCES INC','지스케일러','ETSY INC',
                  'WORKDAY INC','LOWE S COMPANIES INC','BOEING CO','MOTOROLA SOLUTIONS INC','예수금',''],
        '자산종류':['해외주식','해외주식','해외주식','해외주식','해외주식','해외주식','해외주식','해외주식','해외주식',
                    '해외주식','',''],
        '현재가':[58858.32,99856.43,131053.08,134303.77,151595.4,168936.48,213247.08,255122.77,262007.28,
                  316700.27,0,0],
        '목표비중':[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0,1]}
df = pd.DataFrame(data)

row_num = len(df['ISIN코드'])
df.insert(4,'수량',0)
df.insert(5,'평가금액',0)
df.insert(6,'실잔고비중',0)
df.insert(8,'차이',0)
df

,ISIN코드,종목명,자산종류,현재가,수량,평가금액,실잔고비중,목표비중,차이
0,US88339J1051,트레이드 데스크(TTD),해외주식,58858.32,0,0,0,0.1,0
1,US8522341036,블록(스퀘어)(SQ),해외주식,99856.43,0,0,0,0.1,0
2,US2521311074,덱스컴,해외주식,131053.08,0,0,0,0.1,0
3,US64125C1099,NEUROCRINE BIOSCIENCES INC,해외주식,134303.77,0,0,0,0.1,0
4,US98980G1022,지스케일러,해외주식,151595.40,0,0,0,0.1,0
5,US29786A1060,ETSY INC,해외주식,168936.48,0,0,0,0.1,0
6,US98138H1014,WORKDAY INC,해외주식,213247.08,0,0,0,0.1,0
7,US5486611073,LOWE S COMPANIES INC,해외주식,255122.77,0,0,0,0.1,0
8,US0970231058,BOEING CO,해외주식,262007.28,0,0,0,0.1,0
9,US6200763075,MOTOROLA SOLUTIONS INC,해외주식,316700.27,0,0,0,0.1,0


In [134]:
money = 5_000_000

In [135]:
def calc_table():
    row_num = len(df['ISIN코드'])
    df['평가금액'][:-2] = df['현재가'][:-2]*df['수량'][:-2]
    df['평가금액'][row_num-2] = money - np.sum(df['평가금액'][:-2])
    df['실잔고비중'][:-1] = df['평가금액'][:-1]/money
    df['차이'][:-1] = abs(df['실잔고비중'][:-1]-df['목표비중'][:-1])
    df['평가금액'][row_num-1] = np.sum(df['평가금액'][:-1])
    df['실잔고비중'][row_num-1] = np.sum(df['실잔고비중'][:-1])
    df['차이'][row_num-1] = np.sum(df['차이'][:-1])
    
# 목표비중과 실잔고비중의 차이의 절대값 합
def min_gap(수량):
    row_num = len(수량)
    수량 = np.array(수량)
    평가금액 = df['현재가'][:-2] * 수량
    if  money > np.sum(평가금액):
        평가금액[row_num] = money - np.sum(평가금액)
        gap = np.sum(abs(평가금액/money - df['목표비중'][:-1]))
    else:
        gap = 2
    return gap

In [136]:
df['수량'][:-2] = ((money * df['목표비중'][:-2]) / df['현재가'][:-2]).astype(int)
calc_table()
df

,ISIN코드,종목명,자산종류,현재가,수량,평가금액,실잔고비중,목표비중,차이
0,US88339J1051,트레이드 데스크(TTD),해외주식,58858.32,8,470866.56,0.094173,0.1,0.005827
1,US8522341036,블록(스퀘어)(SQ),해외주식,99856.43,5,499282.15,0.099856,0.1,0.000144
2,US2521311074,덱스컴,해외주식,131053.08,3,393159.24,0.078632,0.1,0.021368
3,US64125C1099,NEUROCRINE BIOSCIENCES INC,해외주식,134303.77,3,402911.31,0.080582,0.1,0.019418
4,US98980G1022,지스케일러,해외주식,151595.40,3,454786.20,0.090957,0.1,0.009043
5,US29786A1060,ETSY INC,해외주식,168936.48,2,337872.96,0.067575,0.1,0.032425
6,US98138H1014,WORKDAY INC,해외주식,213247.08,2,426494.16,0.085299,0.1,0.014701
7,US5486611073,LOWE S COMPANIES INC,해외주식,255122.77,1,255122.77,0.051025,0.1,0.048975
8,US0970231058,BOEING CO,해외주식,262007.28,1,262007.28,0.052401,0.1,0.047599
9,US6200763075,MOTOROLA SOLUTIONS INC,해외주식,316700.27,1,316700.27,0.063340,0.1,0.036660


In [158]:
def main_proc(target_gap, bnds):
    
    print("target gap: ", target_gap)
    print("bounds: ", bnds)
    print("len bounds: ", len(bnds))

    global value1, best_var
    var_cnt = len(bnds)
    value1 = 100
    best_var = []
    mutation_rate = 0.3

    # 십진수로 변환해서 목적함수에 전달
    def call_func(binary):
        global value1, best_var
        var = []
        for ii in range(var_cnt):
            binary0 = binary[ii * 8 : ii * 8 + 8]
            decimal = int(binary0, 2) + bnds[ii][0]
            if decimal > bnds[ii][1]:  # 변수값 범위를 벗어나는 경우 오류로 뱉어 냄
                return 100
            var.append(decimal)
        #        print(var)
        result = min_gap(var)
        #        print(result)
        if value1 > result:
            value1 = result
            best_var = var
        return result

    equal_cnt_k = 0
    min_val_k = value1
    for k in range(100):
        chromosome_list = []
        for i in range(32):
            chromosome = ""
            for ii in range(var_cnt):
                decimal = random.randint(0, bnds[ii][1] - bnds[ii][0])
                gene = format(decimal, "b").zfill(8)  # 숫자가 아닌 텍스트로 전환
                chromosome = chromosome + gene  # 주욱 이어붙임
            # print("chromosome: ", chromosome)
            # print("len chron: ", len(chromosome))
            chromosome_list.append(chromosome)

        print(chromosome_list, k)
        equal_cnt_i = 0
        min_val = value1
        for i in range(100):
            #            print("실행횟수 =", k*100+i)
            chromosome_list0 = sorted(chromosome_list, key=call_func)

            if value1 > target_gap:
                if min_val > value1:
                    min_val = value1
                    equal_cnt_i = 0
                else:
                    equal_cnt_i += 1
                if equal_cnt_i >= 30:  # 더 이상 최저값이 갱신되지 않으면 종료한다.
                    break
            else:
                break

            chromosome_list0[-1] = chromosome_list0[0]  # 1등 염색체를 꼴찌 염색체에 엎어친다. 꼴찌는 사라지고 1등은 생존 확률이 높아진다

            # crossover
            # X = random.randint(0, ((var_cnt) * 8))
            # X = X - (X // 8)
            X = random.randint(1, 10) * 8
            if X in [0, 80]: continue
            print(4, ((var_cnt) * 8) - 4, X)
            print("len sorted chromosom", len(chromosome_list0))
            for ii in range(15):
                temp = chromosome_list0[ii][X:]
                print("cutted chromsome segment", temp)
                chromosome_list0[ii] = (
                    chromosome_list0[ii][:X] + chromosome_list0[29 - ii][X:]
                )
                chromosome_list0[29 - ii] = chromosome_list0[29 - ii][:X] + temp
            temp = chromosome_list0[31][X:]
            chromosome_list0[31] = chromosome_list0[31][:X] + chromosome_list0[30][X:]
            chromosome_list0[30] = chromosome_list0[30][:X] + temp

            # mutation
            rand = random.random()
            if rand < mutation_rate:
                for j in range(4):  # 1/8에 돌연변이 염색체를 반영한다.
                    chromosome1 = ""
                    for ii in range(var_cnt):
                        decimal = random.randint(0, bnds[ii][1] - bnds[ii][0])
                        gene = format(decimal, "b").zfill(8)  # 숫자가 아닌 텍스트로 전환
                        chromosome1 = chromosome1 + gene  # 주욱 이어붙임
                    rand_num = random.randint(1, 30)
                    chromosome_list0[rand_num] = chromosome1
            chromosome_list = chromosome_list0

        if min_val_k > min_val:
            min_val_k = min_val
            equal_cnt_k = 0
        else:
            equal_cnt_k += 1
        if equal_cnt_k >= 20:  # 더 이상 최고값이 갱신되지 않으면 종료한다.
            break
    #    print("결과값 = ",value1)
    #    print(best_var)
    df["수량"][:-2] = best_var
    calc_table()
    
df["수량"][:-2] = ((money * df["목표비중"][:-2]) / df["현재가"][:-2]).astype(int)
# 절사 수량에서 +- 3수량한 것을 범위값으로 줌
bnds = tuple((max(1, df["수량"][x] - 3), df["수량"][x] + 3) for x in range(row_num - 2))
target_gap = 0.11  # 괴리율의 합이 20%미만이 목표임
main_proc(target_gap, bnds)
print(df)

target gap:  0.11
bounds:  ((5, 11), (2, 8), (1, 6), (1, 6), (1, 6), (1, 5), (1, 5), (1, 4), (1, 4), (1, 4))
len bounds:  10
['00000110000000110000001100000101000000110000001000000001000000000000001000000011', '00000101000001010000000100000010000001000000001100000000000000010000000000000001', '00000011000001010000010100000100000000110000010000000001000000100000001000000010', '00000001000000010000010100000011000000100000001100000001000000010000000000000010', '00000110000001010000010100000011000001000000001000000011000000100000000100000010', '00000100000001010000001100000010000001000000000000000100000000010000001100000000', '00000010000000110000010000000101000000010000001100000000000000100000001000000000', '00000110000000110000010100000010000000000000000000000011000000100000001000000001', '00000110000001100000001100000011000001010000001100000011000000000000000100000011', '00000010000000110000000100000000000000000000000000000100000000010000000100000000', '000001100000011000000010000000100

In [143]:
format(6, "b")

'110'

In [144]:
3 * 10

30